# Import

In [33]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import Sequential
import seaborn as sns
sns.set()

# Load Data

In [34]:
train=pd.read_csv("/Users/achilleraffin-marchetti/Desktop/data_scientist_project/Projects/digit_recognizer/Datasets/train.csv")
test=pd.read_csv("/Users/achilleraffin-marchetti/Desktop/data_scientist_project/Projects/digit_recognizer/Datasets/test.csv")

In [35]:
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:

test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
X = train.iloc[:, 1:]

In [38]:
y = train.iloc[:,0]

In [39]:
X_test = test.iloc[:,:]

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=365)

# Preprocessing

In [41]:
preprocess=lambda x:(x-127.5)/127.5 

train=tf.data.Dataset.from_tensor_slices(((preprocess(X_train)),(y_train)))
valid=tf.data.Dataset.from_tensor_slices(((preprocess(X_valid)),(y_valid)))

test=tf.data.Dataset.from_tensor_slices((preprocess(X_test)))

In [42]:
train_images=train.map(lambda x,y:(tf.reshape(x,(28,28,1)),y))
valid_images=valid.map(lambda x,y:(tf.reshape(x,(28,28,1)),y))

In [43]:
train_final=train_images.shuffle(X_train.shape[0]).batch(256).cache().prefetch(tf.data.AUTOTUNE)
valid_final=valid_images.batch(256).cache().prefetch(tf.data.AUTOTUNE)

# Model

In [44]:
model = tf.keras.Sequential(
    [
        layers.Input((28,28,1),name="Inputs"),
        layers.Lambda(lambda x: (x+1)/2,name="Lambda"),
        layers.RandomRotation(factor=0.1),
        layers.RandomContrast(factor=0.1),
        layers.Conv2D(32,3, activation="relu", name="Conv_Layer"),
        layers.SpatialDropout2D(0.05,name="Dropout1"),
        layers.MaxPooling2D(2,name="Pooling_layer2"),
        layers.Conv2D(64,3, activation="relu", name="Conv_Layer2"),
        layers.SpatialDropout2D(0.05,name="Dropout2"),
        layers.MaxPooling2D(2,name="Pooling_layer"),
        layers.GlobalAveragePooling2D(name="avgPooling"),
        layers.Dense(64,activation="relu",name="Dense_layer"),
        layers.Dense(10,activation="softmax",name="outputs"),
    ]
)



In [45]:
model.compile(optimizer="Adam",loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=["accuracy"])

In [46]:
model.fit(train_final, validation_data=valid_final, epochs=200)

Epoch 1/200
132/132 [==============================] - 49s 314ms/step - loss: 2.0712 - accuracy: 0.2479 - val_loss: 1.5586 - val_accuracy: 0.4439
Epoch 2/200
132/132 [==============================] - 44s 331ms/step - loss: 1.6131 - accuracy: 0.4403 - val_loss: 1.1165 - val_accuracy: 0.6665
Epoch 3/200
132/132 [==============================] - 43s 323ms/step - loss: 1.3687 - accuracy: 0.5362 - val_loss: 0.8920 - val_accuracy: 0.7383
Epoch 4/200
132/132 [==============================] - 38s 287ms/step - loss: 1.2154 - accuracy: 0.5962 - val_loss: 0.7494 - val_accuracy: 0.7832
Epoch 5/200
132/132 [==============================] - 38s 292ms/step - loss: 1.0954 - accuracy: 0.6334 - val_loss: 0.6387 - val_accuracy: 0.8274
Epoch 6/200
132/132 [==============================] - 37s 282ms/step - loss: 0.9855 - accuracy: 0.6794 - val_loss: 0.5417 - val_accuracy: 0.8615
Epoch 7/200
132/132 [==============================] - 32s 244ms/step - loss: 0.8824 - accuracy: 0.7152 - val_loss: 0.4702 -

132/132 [==============================] - 42s 320ms/step - loss: 0.3108 - accuracy: 0.9065 - val_loss: 0.1631 - val_accuracy: 0.9476
Epoch 37/200
132/132 [==============================] - 48s 363ms/step - loss: 0.3024 - accuracy: 0.9079 - val_loss: 0.1486 - val_accuracy: 0.9521
Epoch 38/200
132/132 [==============================] - 43s 329ms/step - loss: 0.2924 - accuracy: 0.9116 - val_loss: 0.1523 - val_accuracy: 0.9508
Epoch 39/200
132/132 [==============================] - 38s 292ms/step - loss: 0.2881 - accuracy: 0.9106 - val_loss: 0.1436 - val_accuracy: 0.9538
Epoch 40/200
132/132 [==============================] - 31s 235ms/step - loss: 0.2836 - accuracy: 0.9130 - val_loss: 0.1389 - val_accuracy: 0.9557
Epoch 41/200
132/132 [==============================] - 30s 229ms/step - loss: 0.2821 - accuracy: 0.9136 - val_loss: 0.1432 - val_accuracy: 0.9527
Epoch 42/200
132/132 [==============================] - 31s 231ms/step - loss: 0.2760 - accuracy: 0.9164 - val_loss: 0.1390 - val_a

Epoch 92/200
132/132 [==============================] - 32s 245ms/step - loss: 0.1483 - accuracy: 0.9545 - val_loss: 0.0778 - val_accuracy: 0.9754
Epoch 93/200
132/132 [==============================] - 31s 238ms/step - loss: 0.1493 - accuracy: 0.9538 - val_loss: 0.0788 - val_accuracy: 0.9754
Epoch 94/200
132/132 [==============================] - 30s 228ms/step - loss: 0.1503 - accuracy: 0.9541 - val_loss: 0.0773 - val_accuracy: 0.9756
Epoch 95/200
132/132 [==============================] - 31s 237ms/step - loss: 0.1436 - accuracy: 0.9559 - val_loss: 0.0782 - val_accuracy: 0.9743
Epoch 96/200
132/132 [==============================] - 32s 243ms/step - loss: 0.1451 - accuracy: 0.9553 - val_loss: 0.0730 - val_accuracy: 0.9761
Epoch 97/200
132/132 [==============================] - 31s 238ms/step - loss: 0.1432 - accuracy: 0.9552 - val_loss: 0.0783 - val_accuracy: 0.9736
Epoch 98/200
132/132 [==============================] - 31s 232ms/step - loss: 0.1460 - accuracy: 0.9557 - val_loss: 0

132/132 [==============================] - 39s 296ms/step - loss: 0.1067 - accuracy: 0.9680 - val_loss: 0.0616 - val_accuracy: 0.9802
Epoch 148/200
132/132 [==============================] - 31s 235ms/step - loss: 0.1096 - accuracy: 0.9654 - val_loss: 0.0588 - val_accuracy: 0.9820
Epoch 149/200
132/132 [==============================] - 34s 262ms/step - loss: 0.1065 - accuracy: 0.9671 - val_loss: 0.0589 - val_accuracy: 0.9817
Epoch 150/200
132/132 [==============================] - 32s 242ms/step - loss: 0.1081 - accuracy: 0.9667 - val_loss: 0.0598 - val_accuracy: 0.9835
Epoch 151/200
132/132 [==============================] - 30s 226ms/step - loss: 0.1061 - accuracy: 0.9669 - val_loss: 0.0581 - val_accuracy: 0.9826
Epoch 152/200
132/132 [==============================] - 31s 232ms/step - loss: 0.1042 - accuracy: 0.9681 - val_loss: 0.0612 - val_accuracy: 0.9806
Epoch 153/200
132/132 [==============================] - 32s 240ms/step - loss: 0.1027 - accuracy: 0.9681 - val_loss: 0.0605 -

In [47]:
test_images=test.map(lambda x:(tf.reshape(x,(28,28,1))))

In [48]:
y_predict=model.predict(test_images.batch(1024))

28/28 [==============================] - 4s 134ms/step


In [49]:
predict=np.argmax(y_predict,axis=-1)

In [50]:
result_df=pd.read_csv("/Users/achilleraffin-marchetti/Desktop/data_scientist_project/Projects/digit_recognizer/Datasets/sample_submission.csv")
result_df["Label"]=predict
result_df.to_csv("result.csv",index=None, sep=',')